In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import scipy as stats
import statistics
from numpy import mean
#from statistics import mean, median

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc

In [4]:
# Path to sqlite
database_path = "Resources/hawaiinot8.sqlite"     # Creates the path variable

In [5]:
# Create engine to that can talk to the database hawaii.sqlite
engine = create_engine(f"sqlite:///{database_path}")
#engine     # To view engine contents
conn1 = engine.connect()     # Create a connection
conn1     # Shows connection

In [6]:
# Can I see the station table?
engine.execute('SELECT * FROM station LIMIT 3').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0)]

In [7]:
# Checking column headings and data types for station table.
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [8]:
# Can I see the measurement table?
engine.execute('SELECT * FROM measurement LIMIT 3').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0)]

In [9]:
# Checking columns and data types for measurement table.
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [10]:
data = engine.execute("SELECT * FROM station LIMIT 3;")     # First table, station listed
for record in data:
    print(record)

(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)
(2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6)
(3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0)


In [11]:
# Create a dataframe for database records
stationloc = pd.read_sql("SELECT * FROM station", conn1)

In [12]:
# Display the dataframe headers and data
stationloc.head(3)

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0


In [13]:
# Query all records in the hawaii.sqlite database
data = engine.execute("SELECT * FROM measurement LIMIT 3;")     # Second table, measurement listed
for record in data:     # Shows the data in the table
    print(record)

(1, 'USC00519397', '2010-01-01', 0.08, 65.0)
(2, 'USC00519397', '2010-01-02', 0.0, 63.0)
(3, 'USC00519397', '2010-01-03', 0.0, 74.0)


In [14]:
conn2 = engine.connect()     # Create another table connection
conn2     # Shows second connection address

In [15]:
# Create a dataframe for measurement table records
rainmeasure = pd.read_sql("SELECT * FROM measurement", conn2)

In [16]:
# Display the measurement dataframe headers and data
rainmeasure.head(3)

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0


In [17]:
# Declaring the base automatically
base = automap_base()     # Case sensitive

In [18]:
# Use the Base class to reflect the database tables
base.prepare(autoload_with=engine)     # Created engines to use above, does not work with Python3

In [19]:
# Print all of the classes mapped to the base
base.classes.keys()

['measurement', 'station']

In [ ]:
# reflect an existing database into a new model

# reflect the tables


In [ ]:
# View all of the classes that automap found
base.classes.keys()

In [ ]:
Measurement = base.classes.measurement     # Case sensitive

In [ ]:
Station = base.classes.station

In [ ]:
# Save references to each table
# station is common to both tables

# Use a DataFrame?


In [ ]:
combined_data = pd.merge(rainmeasure, stationloc, on="station", how="inner")
combined_data.head(3)

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Query all of the dates - will this work? Case sensitive.
session.query(func.count(Measurement.date)).all()

In [ ]:
# Earliest date query.
session.query(Measurement.date).order_by(Measurement.date).first()

In [ ]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

session.query(Measurement.date).filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()

In [ ]:
# Perform a query to retrieve the data and precipitation scores
query_rain = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Rain date query start: ", query_rain)

In [ ]:
#session.query(Measurement.date, Measurement.prcp).filter(Measurement.date == query_rain).all()

In [ ]:
session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()

In [ ]:
yearly_rainfall = session.query(Measurement.date, Measurement.prcp).\
filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()
yearly_rainfall

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date
# column.

yearly_rainfall_df = pd.DataFrame(yearly_rainfall)
yearly_rainfall_df.head(3)

In [ ]:
# Setting the date column as the index of the DataFrame.
yearly_rainfall_df = pd.DataFrame(yearly_rainfall, columns=['data','prcp'])
yearly_rainfall_df.set_index('data', inplace=True)     # Not unique?
yearly_rainfall_df.head(3)

In [ ]:
# Sort the DataFrame values by "date".     # data already appears to be sorted by date???
yearly_rainfall_df.sort_values(by='data',ascending=True)   # data looks same as before
yearly_rainfall_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

x_axis = np.arange(len(yearly_rainfall_df))
tick_locations = [value for value in x_axis]

#plt.bar(x_axis, df["prcp"])     # Doesn't match lesson graph (very thin lines)

yearly_rainfall_df.plot(legend=False)
plt.xticks(tick_locations, fontsize=8, rotation=90)
plt.title("Hawaiin Rainfall for 1-Year Period", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Precipitation (inches)", fontsize=12)
plt.savefig("Hawaii_Rain.png")     # Blank (white) image if done after plt.show. Change dimensions.
plt.show

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
yearly_rainfall_df.describe()

In [ ]:
precipitation = yearly_rainfall_df['prcp']
mean_numpy = np.mean(precipitation)
print(f"The average rainfall for time period is {mean_numpy} inches.")

In [ ]:
var_numpy = np.var(precipitation,ddof = 0)
print(f"The rainfall variance using the NumPy module is {var_numpy}.")

In [ ]:
sd_numpy = np.std(precipitation,ddof = 0)
print(f"The population standard deviation using the NumPy module is {sd_numpy}.")

In [ ]:
# Need variables converted from text string (from printing statements above) back to float values.
# Need a bottom limit as zero should be the bottom, lowest value. No negative values.

print(f"Roughly 68% of the data is between {round(float(mean_numpy-sd_numpy),3)} and {round(float(mean_numpy+sd_numpy),3)}")
print(f"Roughly 95% of the data is between {round(mean_numpy-2*sd_numpy,3)} and {round(mean_numpy+2*sd_numpy,3)}")
print(f"Roughly 99.7% of the data is between {round(mean_numpy-3*sd_numpy,3)} and {round(mean_numpy+3*sd_numpy,3)}")

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
# use count

session.query(func.count(Station.station)).all()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
# use count and desc

sel = [Measurement.station,
      func.count(Measurement.station)]
station_readings = session.query(*sel).\
    group_by(Measurement.station).\
    order_by(Measurement.station.desc()).all()
station_readings

# Use first() for most active after sorting descending order?

#station_query = session.query(Measurement.station,func.count(Measurement.station)).group_by(Measurement.station).all()
#station_query

# USC00519281 has 2772 readings.

In [ ]:
# Using the most active station id from the previous query, calculate the
# lowest, highest, and average temperature.
# descriptive?

Station_281_temps = session.query(Measurement.station, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').all()
Station_281_temps



In [ ]:
mean = Station_281_temps[Measurement.tobs].mean()

In [ ]:
#temps = [Station_281_temps,     # Not working?
        #func.avg(Measurement.tobs),
        #func.min(Measurement.tobs),
        #func.max(Measurement.tobs)]

#temps

In [ ]:
session.query(func.max(Measurement.tobs)).all()

In [ ]:
# Average temperature for Station USC00519281.
# There are null readings for some rows. Will row count be accurate or strip null rows first?

ave_281_temp = statistics.mean(Station_281_temps)
ave_281_temp

In [ ]:
# Minimum temperature for Station USC00519281.
min_281_temp = min(Station_281_temps)
min_281_temp

In [ ]:
# Maximum temperature for Station USC00519281.
max_281_temp = max(Station_281_temps)
max_281_temp

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


In [ ]:
# From above
rainmeasure.head(3)

In [ ]:
Station_281_tempsB = session.query(Measurement.station, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').filter(Measurement.date > '2016-08-23').all()
Station_281_tempsB

In [ ]:
type(Station_281_tempsB)

In [ ]:
Station_281_tempsC = Station_281_tempsB.astype(float)

In [ ]:
filterna_281_list = []
for tob in Station_281_tempsB:
    if type(Measurement.tobs) == float:
        filterna_281_list.append(Measurement.tobs)

In [ ]:
mean(Station_281_tempsB)

In [ ]:
Station_281_ytempsC = session.query(Measurement.date, Measurement.prcp).\
filter(Measurement.date > '2016-08-23').order_by(Measurement.date).all()
Station_281_ytempsC

In [ ]:
yearly_data = session.query(Measurement.date,
    Measurement.station, Measurement.tobs).\
        filter[(Measurement.date > '2016-08-23') AND (Measurement.station == 'USC00519281')]
yearly_data

In [ ]:
station_281_temps = session.query(yearly_data,
    Measurement.station, Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').all()
station_281_temps

# Close session

In [ ]:
# Close Session
session.close()